In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [73]:
f = pd.read_csv('eurostat/Water_Use_Data_RBD.csv', )
d = pd.read_csv('eurostat/Treatment_plants_by_type_of_treatment_and_RBD.csv')
#fD = f[f['RBD'].str.contains("DE")]
#set(fD.WAT_PROC)
years = np.arange(2000,2016)
rbd = 'DE'
vals = ['d','f']
for t in years:
    all_ = True
    for v in vals:
        x = vars()[v]
        x = x[(x.TIME==t) & (x.RBD.str.contains(rbd))]
        vars()[v+str(t)[2:]] = x.set_index(['RBD','TIME'])
        if x[x.Value != ':'].size == 0:
            all_ = False
    if all_ == True:
        print('Got data for {}'.format(t))
    else: all_ = True

Got data for 2007
Got data for 2010


In [74]:
f07.Value = f07.Value.astype(float)
pd.get_dummies(f07)

Value  \
RBD                     TIME             
Danube - RBD (DE)       2007      0.06   
                        2007   3208.43   
                        2007     20.16   
                        2007    465.63   
                        2007   3694.28   
Rhine - RBD (DE)        2007     48.72   
                        2007  12197.27   
                        2007     34.16   
                        2007   1691.38   
                        2007  13971.53   
Moselle-Saar (DE)       2007      0.07   
                        2007    174.15   
                        2007      0.86   
                        2007     80.77   
                        2007    255.84   
Ems - RBD (DE)          2007      0.65   
                        2007    195.43   
                        2007      0.23   
                        2007    132.48   
                        2007    328.79   
Elbe / Labe - RBD (DE)  2007     38.23   
                        2007   5477.32   
                        2007     70.12   
                        2007    694.72   
                        2007   6280.40   
Oder / Odra - RBD (DE)  2007      0.23   
                        2007     51.78   
                        2007      0.21   
                        2007     24.45   
                        2007     76.68   
Maas / Meuse - RBD (DE) 2007      2.55   
                        2007    113.97   
                        2007      1.00   
                        2007     88.65   
                        2007    206.17   

                              WAT_PROC_Public water supply and self and other supply  \
RBD                     TIME                                                           
Danube - RBD (DE)       2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
Rhine - RBD (DE)        2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
Moselle-Saar (DE)       2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
Ems - RBD (DE)          2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
Elbe / Labe - RBD (DE)  2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
                        2007                                                  1        
Oder / Odra - RBD (DE)  2007                                                  1        
              

In [10]:
fD = fD[fD.Value != ':']
fD.Value = fD.Value.astype(float)

In [11]:

for rbd in fD.RBD.values[0]:
    df = fD[fD.RBD == rbd]
    for type_ in df.WAT_PROC.values:
        ddf = df[df.WAT_PROC.values == type_]
        #print(ddf[['TIME','Value']].describe())
        ddf.plot(x='TIME',y='Value')
        plt.title('Region: {}, {}'.format(rbd,type_))
        

In [ ]:
precip.plot(x='TIME',y='')